<a href="https://colab.research.google.com/github/Nukaraju2003/DR.Abhinav-Dayal-Sir/blob/main/speakerdiarizationusingagglomerativeclustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 the language being spoken is English, then change language to 'English' as this improves performance.


1.   I'm using Open AI's Whisper model to seperate audio into segments and generate transcripts.
2.   I'm then generating speaker embeddings for each segments.
3.   Then I'm using agglomerative clustering on the embeddings to identify the speaker for each segment.   

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install soundfile

In [5]:
import librosa
import soundfile as sf
import numpy as np


def load_audio(audio_file_path, mono=True):
    # Load the audio file using librosa
    audio_data, sr = librosa.load(audio_file_path, sr=None, mono=mono)
    return audio_data, sr

# Replace 'your_audio_file.wav' with the path to your audio file
audio_file_path = '/content/drive/MyDrive/Jyothi Mam/student1.wav'

# Load the audio data with multiple channels
audio_data, sample_rate = load_audio(audio_file_path, mono=False)
if len(audio_data.shape) > 1 and audio_data.shape[0] > 1:
    path = audio_data.mean(axis=0)
else:
    path = audio_data


if isinstance(path, np.ndarray):
    # Save processed audio to a WAV file
    sf.write('processed_audio.wav', path, sample_rate)
    path = 'processed_audio.wav'  # Update the 'path' variable to the file path

# Now, use 'path' for FFmpeg conversion if necessary
if path[-3:] != 'wav':
    subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
    path = 'audio.wav'


In [6]:
num_speakers = 2 #@param {type:"integer"}

language = 'English' #@param ['any', 'English']

model_size = 'large-v2' #@param ['tiny', 'base', 'small', 'medium', 'large', 'large-v2']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


In [7]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [8]:
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 2.87G/2.87G [01:01<00:00, 50.1MiB/s]


In [9]:
result = model.transcribe(path)
segments = result["segments"]

In [10]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [11]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [12]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [13]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [14]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [15]:
# import wave

# def get_audio_channels(audio_file_path):
#     with wave.open(audio_file_path, 'rb') as audio_file:
#         num_channels = audio_file.getnchannels()
#         return num_channels

# # Replace 'your_audio_file.wav' with the path to your audio file
# audio_file_path = '/content/drive/MyDrive/Jyothi Mam/student1.wav'
# num_channels = get_audio_channels(audio_file_path)

# print(f"The number of channels in the audio file '{audio_file_path}' is: {num_channels}")
